In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install --upgrade pip

In [4]:
# Carica il modulo CUDA 11.7
#!module load cuda/11.7

In [5]:
#!pip install shap

In [6]:
#!pip install numpy --upgrade

In [7]:
#!pip install numpy --upgrade pip

In [8]:
#!pip install pandas --upgrade

In [9]:
#!pip uninstall pandas -y
#!pip install pandas

In [10]:
#!pip install pycocotools

In [11]:
#pip install "numpy<2.0.0"


In [12]:
##Imports
import numpy as np 
import torch
import os, sys, argparse, datetime, shutil
import torchvision.models as models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tensorflow import keras
from tensorflow.python.keras.models import model_from_json
from keras.utils import to_categorical
import shap 
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

#Import from current folder
from utils.config import *
from utils.dataloader import *
from utils.engine import train_one_epoch, evaluate
from utils.train import compute_json_detection
from utils.coco_utils import get_coco_api_from_dataset
from utils.coco_eval import CocoEvaluator
from utils.knowledge_graph import compare_shap_and_KG, reduce_shap, GED_metric, get_bbox_weight
import utils.utils as uti

2024-11-08 15:51:39.476740: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-08 15:51:39.491806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731077499.509812 2076919 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731077499.515331 2076919 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 15:51:39.535091: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [13]:
os.environ["TF_XLA_FLAGS"] = ""

In [14]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [15]:
#os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/usr/local/cuda-11.7"
#os.environ["PATH"] = "/usr/local/cuda-11.7/bin:" + os.environ["PATH"]
#os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-11.7/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")

In [16]:
!which ptxas
!nvcc --version

/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages/triton/third_party/cuda/bin/ptxas
/bin/bash: line 1: nvcc: command not found


In [17]:
#pip install tools

In [18]:
sys.path

['/usr/lib64/python39.zip',
 '/usr/lib64/python3.9',
 '/usr/lib64/python3.9/lib-dynload',
 '',
 '/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages',
 '/orfeo/cephfs/home/dssc/galessio/jupyter/lib/python3.9/site-packages',
 '../monumai']

In [19]:
#pip uninstall -y numpy tensorflow

In [20]:
#pip install numpy==1.24 tensorflow

In [21]:
import argparse
import sys
import os

# Imposta il percorso della directory manualmente
os.chdir(os.path.expanduser('~/XAI_Monuments/tools'))
notebook_dir = os.getcwd()
sys.path.append(notebook_dir)

from pickle_tools import *

TMP_TRAIN = TMP_PATH + '/train'
TMP_VAL = TMP_PATH + '/val'
TMP_TEST = TMP_PATH + '/test'

os.makedirs(TMP_VAL, exist_ok=True)
os.makedirs(TMP_TRAIN, exist_ok=True)
os.makedirs(TMP_TEST, exist_ok=True)

# Argparse
parser = argparse.ArgumentParser(description='Arguments needed to prepare the metadata files')
parser.add_argument('--resume', dest='resume', help='Whether or not to resume a training', default=False)
parser.add_argument('--path_resume', dest='path_resume', help='Path to the model to load', default='./model/model_monumenai.pth')
parser.add_argument('--epoch_classif', dest='epoch_classif', help='Number of epochs to train the classification model', default=150)
parser.add_argument('--batch_size', dest='batch_size', help='Batch size to train the classification model', default=8)
parser.add_argument('--neuron_classif', dest='neuron_classif', help='Number of neurons in the classification model', default=11)
parser.add_argument('--epoch_detection', dest='epoch_detection', help='Number of epochs to train the detection model', default=0)
parser.add_argument('--lr', dest='lr', help='Learning rate of the detection model', default=0.0003)
parser.add_argument('--stepLR', dest='stepLR', help='Step of the learning rate scheduler', default=9)
parser.add_argument('--gammaLR', dest='gammaLR', help='Gamma parameter of the learning rate scheduler', default=0.1)
parser.add_argument('--weight', dest='weight', help='Type of weighting', default='None')
parser.add_argument('--exp_weights', dest='exp_weights', help='linear or exponential weighting', default='linear')
parser.add_argument('--data', dest='data', help='MonumenAI or PascalPart', default='MonumenAI')

# Se stai usando un ambiente Jupyter, ignora gli argomenti aggiuntivi
if 'ipykernel_launcher' in sys.argv[0]:
    sys.argv = sys.argv[:1]

args = parser.parse_args()

In [22]:
#Hyperparameters classification
n_neurons_classification = int(args.neuron_classif)
num_epochs_classification = int(args.epoch_classif)
batch_size_classification = int(args.batch_size)
learning_rate_classification = None

#Hyperparameters detection
args.epoch_detection = 20
num_epochs_detection = int(args.epoch_detection)
learning_rate_detection = float(args.lr)
stepLR = float(args.stepLR)
gammaLR = float(args.gammaLR)
data = args.data

In [23]:
import os
import argparse
import sys
os.chdir(os.path.expanduser('~/XAI_Monuments/tools'))
notebook_dir = os.getcwd()
sys.path.append(notebook_dir)
if data == 'MonumenAI':
    from metadata_tools import *
    from monumai.monument import Monument
    archi_features = [el for sublist in list(Monument.ELEMENT_DIC.values()) for el in sublist]
    styles = FOLDERS_DATA
    #Loaders for detection
    PATH_DATA = "../"+PATH_DATA
    train_loader = ArchitectureDetectionDataset(os.path.join(PATH_DATA, 'train.csv'), os.path.join(PATH_DATA, CSV_XML), transform_detection)

    if num_epochs_detection != 0:
        val_loader = ArchitectureDetectionDataset(os.path.join(PATH_DATA,'val.csv'), os.path.join(PATH_DATA, CSV_XML), transform_detection)
    else:
        #Actually loading the test set
        val_loader = ArchitectureDetectionDataset(os.path.join(PATH_DATA,'val.csv'), os.path.join(PATH_DATA, CSV_XML), transform_detection)
        test_loader = ArchitectureDetectionDataset(os.path.join(PATH_DATA,'test.csv'), os.path.join(PATH_DATA, CSV_XML), transform_detection)
if data == 'PascalPart':
    from tools.metadata_tools_pascal import *
    from monumai.pascal import Monument
    archi_features = [el for sublist in list(Monument.ELEMENT_DIC.values()) for el in sublist]
    styles = list(PASCAL_EL_DIC.keys())
    #Loaders for detection
    train_loader = PascalDetectionDataset(train_pascal, PATH_PASCAL+PASCAL_IMG,PATH_PASCAL+PASCAL_XML, transform_detection_pascal)
    if num_epochs_detection != 0:
        val_loader = PascalDetectionDataset(val_pascal, PATH_PASCAL+PASCAL_IMG,PATH_PASCAL+PASCAL_XML, transform_detection_pascal)
    else:
        #Actually loading the test set
        val_loader = PascalDetectionDataset(val_pascal, PATH_PASCAL+PASCAL_IMG,PATH_PASCAL+PASCAL_XML, transform_detection_pascal)
        test_loader = PascalDetectionDataset(test_pascal, PATH_PASCAL+PASCAL_IMG,PATH_PASCAL+PASCAL_XML, transform_detection_pascal)

In [24]:
##Hyperparameters for detection
num_archi_features = len(archi_features)
num_classes_detection = num_archi_features + 1  # num_archi_features + background
num_styles = len(styles)

In [25]:
##Build detection model
if args.weight == "bbox_level":
    from utils.pytorch_utils import fasterrcnn_resnet50_fpn_custom
    detector = fasterrcnn_resnet50_fpn_custom(True)
    in_features = detector.roi_heads.box_predictor.cls_score.in_features
    detector.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes_detection)
else:
    detector = models.detection.fasterrcnn_resnet50_fpn(True)
    in_features = detector.roi_heads.box_predictor.cls_score.in_features
    detector.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes_detection)

if args.exp_weights == 'exponential':
    is_exponential = True
elif args.exp_weights == 'linear':
    is_exponential = False
else:
    print("Unrecognized type of weighting, defaulted to linear")
    is_exponential = False


/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
import torch
import torchvision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
detector.to(device)

optimizer = torch.optim.SGD(detector.parameters(), lr=learning_rate_detection, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, stepLR, gamma=gammaLR, last_epoch=-1)

if args.resume:
    detector.load_state_dict(torch.load(args.path_resume, map_location=device))

/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(


In [27]:
import time, json, os
def compute_json_detection(detector, loader, path, dataset = 'MonumenAI'):
    #Run inference on all data to prepare for classification
    detector.eval()
    if dataset == 'MonumenAI':
        information_about_class = ['M', 'G', 'R', 'B']
        elemen_dic = SUB_ELEMENTS
        reverse_dic = SUB_ELEMENTS_REVERSED
    if dataset == 'PascalPart':
        information_about_class = list(PASCAL_EL_DIC.keys())
        elemen_dic = PASCAL_PART_DIC
        reverse_dic = PASCAL_PART_DIC_REVERSED
    for k in range(10):#len(loader)):
        print(f"{k/len(loader)*100}", end='\r')
        img = loader[k][0][0].cuda()
        results = detector([img])[0]
        r_b = results['boxes'].detach().cpu().numpy()
        scores = results['scores'].detach().cpu().numpy()
        classes = results['labels'].detach().cpu().numpy()
        unique, counts = np.unique(classes, return_counts=True)
        counter = dict(zip(unique, counts))

        if dataset == 'MonumenAI':
            img_name = loader.images_loc.iloc[k, 0]
        if dataset == 'PascalPart':
            img_name = loader.images_loc[k]

        condensced_results = {}
        condensced_results["num_predictions"] = []
        condensced_results["image"] = img_name
        condensced_results["object"] = []
        if dataset == 'MonumenAI':
            condensced_results["true_label"] = int(loader.images_loc.iloc[k, 1])
        if dataset == 'PascalPart':
            condensced_results["true_label"] = get_label(loader.xml_loc[k], PASCAL_EL_DIC)

        for name in elemen_dic:

            if elemen_dic[name] in counter:
                condensced_results["num_predictions"].append({
                    name :  int(counter[elemen_dic[name]])
                })
            else:
                condensced_results["num_predictions"].append({
                    name :  0
                })
            
        for k in range(len(r_b)):
            if classes[k] in reverse_dic:
                box = r_b[k]/224.
                local_result = {
                    "bndbox" : {
                        "xmin": str(box[0]),
                        "ymin": str(box[1]),
                        "ymax": str(box[3]),
                        "xmax": str(box[2])
                    },
                    "score" : str(scores[k]),
                    "class" : reverse_dic[classes[k]]
                }
                condensced_results["object"].append(local_result)
        local_path = os.path.join(path,information_about_class[condensced_results["true_label"]] + '_' + img_name.split('/')[-1][:-4] + '.json')
        with open(local_path, 'w') as fp:
            json.dump(condensced_results, fp)

In [28]:
### Loop ~ Epochs. First epoch is regular detection training
for j in range(num_epochs_detection+1):
    print('------------------')
    print("Epoch " + str(j+1) + '/' + str(num_epochs_detection))
    #Detection Hyperparameters
    if j > 0 or args.resume:
        detector.eval()
        #Run inference on all data to prepare for classification
        compute_json_detection(detector, train_loader, TMP_TRAIN,dataset=data)
        compute_json_detection(detector, val_loader, TMP_VAL,dataset=data)
        #If necessary run on test set aswell
        if num_epochs_detection == 0:
            compute_json_detection(detector, test_loader, TMP_TEST,dataset=data)

        print('Inference run')

        #Application of the aggregation function first for train then for val
        matrix_metadata = metadata_to_matrix(TMP_TRAIN, "json")
        names = matrix_metadata[:,-1]
        train_data = np.zeros((len(names),num_archi_features))
        train_label = np.zeros(len(names))
        if data == "MonumenAI":
            for i in range(len(names)):
                im_name = names[i][2:-4]
                idx = train_loader.images_loc['path'].str.contains(im_name)
                train_data[idx] = matrix_metadata[i,:num_archi_features]
                train_label[idx] = matrix_metadata[i,num_archi_features]
        if data == "PascalPart":
            for i in range(len(names)):
                im_name = os.path.join(PATH_PASCAL+PASCAL_IMG,names[i].split('_')[1][:-5] + '.jpg')
                idx = train_loader.images_loc.index(im_name)
                train_data[idx] = matrix_metadata[i,:num_archi_features]
                train_label[idx] = matrix_metadata[i,num_archi_features]
        train_data = train_data.astype(np.float32)
        train_label = to_categorical(train_label.astype(np.float32).astype(np.int8))

        matrix_metadata = metadata_to_matrix(TMP_VAL, "json")
        names = matrix_metadata[:,-1]
        test_data = np.zeros((len(names),num_archi_features))
        test_label = np.zeros(len(names))
        if data == "MonumenAI":
            print(len(names))
            for i in range(len(names)):
                print ("i:",i)
                im_name = names[i][2:-4]
                idx = val_loader.images_loc['path'].str.contains(im_name)
                test_data[idx] = matrix_metadata[i,:num_archi_features]
                test_label[idx] = matrix_metadata[i,num_archi_features]
        if data == "PascalPart":
            for i in range(len(names)):
                im_name = os.path.join(PATH_PASCAL+PASCAL_IMG,names[i].split('_')[1][:-5] + '.jpg')
                idx = val_loader.images_loc.index(im_name)
                test_data[idx] = matrix_metadata[i,:num_archi_features]
                test_label[idx] = matrix_metadata[i,num_archi_features]
        test_data = test_data.astype(np.float32)
        test_label = to_categorical(test_label.astype(np.float32).astype(np.int8))

        #Classification training
        #Initialize model
        print("starting initialization of model")
        classificator = keras.Sequential()
        print("keras.sequential done")
        classificator.add(keras.layers.Dense(units=n_neurons_classification, activation='relu', input_shape=(num_archi_features,)))
        classificator.add(keras.layers.Dense(units=num_styles, activation='softmax'))
        classificator.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        #Train model
        history = classificator.fit(train_data, train_label, batch_size=batch_size_classification, epochs=num_epochs_classification, verbose=0)
        loss, accuracy = classificator.evaluate(test_data, test_label, verbose=1)
        #SHAP
        elements = np.random.choice(len(train_data), int(0.3*len(train_data)), False)
        explainer = shap.KernelExplainer(classificator.predict, train_data[elements])
        #Apply aggregation function to test if necessary
        if num_epochs_detection==0:
            matrix_metadata = metadata_to_matrix(TMP_TEST, "json")
            names = matrix_metadata[:,-1]
            test_data = np.zeros((len(names),num_archi_features))
            test_label = np.zeros(len(names))
            if data == "MonumenAI":
                for i in range(len(names)):
                    im_name = names[i][2:-4]
                    idx = test_loader.images_loc['path'].str.contains(im_name)
                    test_data[idx] = matrix_metadata[i,:num_archi_features]
                    test_label[idx] = matrix_metadata[i,num_archi_features]
            if data == "PascalPart":
                for i in range(len(names)):
                    im_name = os.path.join(PATH_PASCAL+PASCAL_IMG,names[i].split('_')[1][:-5] + '.jpg')
                    idx = test_loader.images_loc.index(im_name)
                    test_data[idx] = matrix_metadata[i,:num_archi_features]
                    test_label[idx] = matrix_metadata[i,num_archi_features]

            test_data = test_data.astype(np.float32)
            non_cat = np.copy(test_label)
            test_label = to_categorical(test_label.astype(np.float32).astype(np.int8))
        shap_values_test = explainer.shap_values(test_data, nsamples=30, l1_reg='bic')
        #Compute GED based on shap 
        d = GED_metric(test_data, shap_values_test, dataset=data)
        print('SHAP GED: ', d)
        if j < num_epochs_detection:
            #Compute relevant shap values & shap weights
            shap_values_train = explainer.shap_values(train_data, nsamples=30, l1_reg='bic')
            labels = np.argmax(train_label,axis=1)
            if args.weight == "instance_level":
                contributions_shap = compare_shap_and_KG(shap_values_train, labels, dataset=data)
                shap_coeff = reduce_shap(contributions_shap,is_exponential)
            elif args.weight == "bbox_level":
                shap_weights = get_bbox_weight(shap_values_train,is_exponential, dataset=data)
            print("Shap computed")
        print('Test loss: ', loss, '\tTest accuracy: ', accuracy)
    if j < num_epochs_detection:
        #Train detection
        metric_logger = uti.MetricLogger(delimiter="  ")
        metric_logger.add_meter('lr', uti.SmoothedValue(window_size=1, fmt='{value:.6f}'))
        header = 'Epoch: [{}]'.format(j+1)

        index = 0
        detector.train()
        #for images, targets in metric_logger.log_every(train_loader, 250, header):
        ###
        max_batches = 5  # or any other small number
        for j, (images, targets) in enumerate(metric_logger.log_every(train_loader, 2, header)):
            if j >= max_batches:
                break
        ###
            images = list(image.to('cuda') for image in images)
            targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]
            #SHAP if necessary
            #-------
            if (j > 0 or args.resume) and args.weight == "bbox_level":
                loss_dict = detector(images, targets, weights=shap_weights[index][:,labels[index]])
                index += 1
            else :
                loss_dict = detector(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            if (j > 0 or args.resume) and args.weight == "instance_level":
                losses = losses * shap_coeff[index]
                index += 1            
            #-------
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            metric_logger.update(loss=losses, **loss_dict)
            metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        scheduler.step()
        #Saving model
        if args.path_resume is None:
            torch.save(detector.state_dict(), DETECTOR_PATH)
        else:
            torch.save(detector.state_dict(), "../"+args.path_resume)
    #Evaluation
    if num_epochs_detection == 0:
        loss, accuracy = classificator.evaluate(test_data, test_label, verbose=1)
        predict_test = classificator(test_data)
        prediction = np.argmax(predict_test,axis=1)
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(non_cat,prediction)
        STYLES_HOTONE_ENCODE = {'M' : 0, 'G' : 1, 'R' : 2, 'B' : 3}
        from sklearn.metrics import ConfusionMatrixDisplay
        import matplotlib.pyplot as plt 

        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                        display_labels=STYLES_HOTONE_ENCODE.keys())
        disp.plot(include_values=True, cmap='viridis')
        plt.savefig('confmatDeLDECAS.png')
        shap_values_test = explainer.shap_values(test_data, nsamples=30, l1_reg='bic')
        #Compute GED based on shap 
        d = GED_metric(test_data, shap_values_test, dataset=data)
        print(d)
        print(accuracy)
    #if j < num_epochs_detection or num_epochs_detection==0:
    if num_epochs_detection==0:
        evaluate(detector, test_loader, device="cuda")

------------------
Epoch 1/20


/orfeo/cephfs/home/dssc/galessio/jupyter/lib64/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Attempt to open cnn_infer failed: handle=0 error: libcudnn_cnn_infer.so.8: cannot open shared object file: No such file or directory (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:81.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch: [1]  [  0/908]  eta: 0:16:35  lr: 0.000300  loss: 0.4098 (0.4098)  loss_classifier: 0.2420 (0.2420)  loss_box_reg: 0.0744 (0.0744)  loss_objectness: 0.0609 (0.0609)  loss_rpn_box_reg: 0.0325 (0.0325)  time: 1.0965  data: 0.0365  max mem: 943
Epoch: [1]  [  2/908]  eta: 0:07:13  lr: 0.000300  loss: 0.4098 (0.6483)  loss_classifier: 0.2420 (0.3943)  loss_box_reg: 0.0744 (0.0898)  loss_objectness: 0.1507 (0.1321)  loss_rpn_box_reg: 0.0325 (0.0321)  time: 0.4780  data: 0.0727  max mem: 1107
Epoch: [1]  [  4/908]  eta: 0:05:40  lr: 0.000300  loss: 0.4098 (0.6866)  loss_classifier: 0.2420 (0.4044)  loss_box_reg: 0.0744 (0.1070)  loss_objectness: 0.1507 (0.1350)  loss_rpn_box_reg: 0.0325 (0.0401)  time: 0.3772  data: 0.1037  max mem: 1107
------------------
Epoch 2/20


/orfeo/cephfs/home/dssc/galessio/jupyter/lib/python3.9/site-packages/albumentations/core/bbox_utils.py:476: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)


Inference run29703
Load .json metadata:  ./tmp/train/B_20181212_110313.json
Load .json metadata:  ./tmp/train/B_5bed71eae40c4.json
Load .json metadata:  ./tmp/train/B_20181212_110321.json
Load .json metadata:  ./tmp/train/M_puerta_mihrab_mezquita_cordoba.json
Load .json metadata:  ./tmp/train/B_5bad4cbc18c44.json
Load .json metadata:  ./tmp/train/R_5c1f82413637d.json
Load .json metadata:  ./tmp/train/G_5c67d991a4cf7.json
Load .json metadata:  ./tmp/train/B_5dd7d6e26f601.json
Load .json metadata:  ./tmp/train/M_alcazaba_malaga3.json
Load .json metadata:  ./tmp/train/G_casa_bailio_cordoba.json
Load .json metadata:  ./tmp/train/G_5babc82fc022f.json
Load .json metadata:  ./tmp/train/M_5e4fb912bf15b.json
Load .json metadata:  ./tmp/train/R_5c75195fbb8f2.json
Load .json metadata:  ./tmp/train/G_5bb8f33edfa1f.json
Load .json metadata:  ./tmp/train/M_5bb0d93bcd653.json
Load .json metadata:  ./tmp/train/B_5c15322219e6f.json
Load .json metadata:  ./tmp/train/B_facultad_derecho_granada2.json
Load

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 302/303 [02:22<00:00,  2.13it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 303/303 [02:22<00:00,  2.12it/s]


ValueError: operands could not be broadcast together with shapes (4,) (17,) 

In [ ]:
shutil.rmtree(TMP_PATH)